
# CS 5588 — Enhanced RAG + Gemini + Fine-Tuning on Online Dataset
_Generated: 2025-09-14T13:53:05_

### 1) Install

In [7]:

!pip -q install -U langchain langchain-community chromadb pypdf             sentence-transformers transformers datasets evaluate peft accelerate tiktoken             langchain-google-genai google-genai
print("If upgraded core libs, consider restarting runtime.")


If upgraded core libs, consider restarting runtime.


### 2) Keys & Imports

In [8]:

import os, getpass, json, sys, platform, pathlib, datetime, importlib, torch
if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass.getpass("Enter your GEMINI_API_KEY: ")
os.environ["GOOGLE_API_KEY"] = os.environ.get("GOOGLE_API_KEY", os.environ["GEMINI_API_KEY"])

from google import genai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

from datasets import load_dataset
import evaluate
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments, pipeline
from peft import LoraConfig, get_peft_model, PeftModel

pathlib.Path("data").mkdir(exist_ok=True)
pathlib.Path("artifacts/ft").mkdir(parents=True, exist_ok=True)
print("Env ready.")


Env ready.


### 3) Env log → env_rag.json

In [9]:

def pv(m):
    try:
        import importlib
        mod = importlib.import_module(m)
        return getattr(mod, "__version__", "unknown")
    except: return "not installed"
env = {
  "timestamp": datetime.datetime.now().isoformat(),
  "python": sys.version, "platform": platform.platform(),
  "cuda_available": torch.cuda.is_available(),
  "packages": {m: pv(m) for m in [
    "langchain","langchain_community","chromadb","tiktoken","transformers",
    "datasets","evaluate","peft","sentence_transformers",
    "langchain_google_genai","google.genai"
  ]}
}
json.dump(env, open("env_rag.json","w"), indent=2)
print(json.dumps(env, indent=2))


{
  "timestamp": "2025-09-19T04:56:20.344545",
  "python": "3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]",
  "platform": "Linux-6.1.123+-x86_64-with-glibc2.35",
  "cuda_available": true,
  "packages": {
    "langchain": "0.3.27",
    "langchain_community": "0.3.29",
    "chromadb": "1.1.0",
    "tiktoken": "0.11.0",
    "transformers": "4.56.1",
    "datasets": "4.1.1",
    "evaluate": "0.4.6",
    "peft": "0.17.1",
    "sentence_transformers": "5.1.0",
    "langchain_google_genai": "unknown",
    "google.genai": "1.38.0"
  }
}


### 4) Upload & Load project docs, Chunk, Build Chroma

In [10]:

# Upload
try:
    from google.colab import files
    up = files.upload()
    import os
    for n,c in up.items():
        open(os.path.join("data", n), "wb").write(c)
    print("Uploaded:", list(up.keys()))
except Exception as e:
    print("Colab upload UI not available.", e)

# Load
import os
def load_docs(folder="data"):
    docs=[]
    for fname in os.listdir(folder):
        p=os.path.join(folder,fname)
        if not os.path.isfile(p): continue
        ext=fname.lower().split(".")[-1]
        try:
            if ext=="pdf": loader=PyPDFLoader(p)
            elif ext in ["txt","md","markdown"]: loader=TextLoader(p, encoding="utf-8")
            else: print("Skip", fname); continue
            docs += loader.load()
        except Exception as e:
            print("Fail", fname, e)
    return docs

raw_docs=load_docs("data")
splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits=splitter.split_documents(raw_docs)

# Chroma
emb = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vs = Chroma.from_documents(splits, embedding=emb, persist_directory="./chroma_minilm")
vs.persist()
retriever = vs.as_retriever(search_kwargs={"k":4})
print("Docs:", len(raw_docs), "Chunks:", len(splits))


Saving 1_NeurIPS-2024-mdagents-an-adaptive-collaboration-of-llms-for-medical-decision-making-Paper-Conference.pdf to 1_NeurIPS-2024-mdagents-an-adaptive-collaboration-of-llms-for-medical-decision-making-Paper-Conference (1).pdf
Saving 2_NeurIPS-2024-richelieu-self-evolving-llm-based-agents-for-ai-diplomacy-Paper-Conference.pdf to 2_NeurIPS-2024-richelieu-self-evolving-llm-based-agents-for-ai-diplomacy-Paper-Conference (1).pdf
Saving 3_NeurIPS-2024-can-large-language-model-agents-simulate-human-trust-behavior-Paper-Conference.pdf to 3_NeurIPS-2024-can-large-language-model-agents-simulate-human-trust-behavior-Paper-Conference (1).pdf
Uploaded: ['1_NeurIPS-2024-mdagents-an-adaptive-collaboration-of-llms-for-medical-decision-making-Paper-Conference (1).pdf', '2_NeurIPS-2024-richelieu-self-evolving-llm-based-agents-for-ai-diplomacy-Paper-Conference (1).pdf', '3_NeurIPS-2024-can-large-language-model-agents-simulate-human-trust-behavior-Paper-Conference (1).pdf']
Docs: 252 Chunks: 2132


### 5) RAG Chains: Gemini & Local FLAN-T5 (pre-FT)

In [11]:

llm_gemini = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)
qa_gemini = RetrievalQA.from_chain_type(llm=llm_gemini, chain_type="stuff", retriever=retriever, return_source_documents=True)

base_model = "google/flan-t5-small"
tok = AutoTokenizer.from_pretrained(base_model)
base = AutoModelForSeq2SeqLM.from_pretrained(base_model)
pipe_base = pipeline("text2text-generation", model=base, tokenizer=tok, max_new_tokens=256, device=0 if torch.cuda.is_available() else -1)
llm_local = HuggingFacePipeline(pipeline=pipe_base)
qa_local = RetrievalQA.from_chain_type(llm=llm_local, chain_type="stuff", retriever=retriever)

def ask(chain, q):
    r=chain({"query": q})
    print("\nQ:", q); print("A:", r.get("result",""))
ask(qa_gemini, "What are two key methods in these documents?")
ask(qa_local, "Summarize one limitation discussed.")


Device set to use cuda:0



Q: What are two key methods in these documents?
A: Based on the provided documents, two key methods or procedures described are:

1.  **Evaluating the accuracy of claims:** A method for assessing whether the main claims made in a paper's abstract and introduction accurately reflect its contributions and scope.
2.  **Using predefined macros for answers:** A procedural method for responding to checklist questions by using specific, provided macros (e.g., `[Yes]`, `[NA]`).

Q: Summarize one limitation discussed.
A: Justification: The paper includes a section on limitations, discussing the scope of the framework, and areas for future improvement. Guidelines: • The answer NA means that the paper has no limitation while the answer No means that the paper has limitations, but those are not discussed in the paper. • The authors are encouraged to create a separate "Limitations" section in their paper. • The authors are encouraged to create a separate "Limitations" section in their paper. • The

### 6) Fine-Tune on online dataset (Hugging Face `squad`, sampled) with LoRA

In [13]:
from datasets import load_dataset
import torch, json
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model

dataset_name="squad"; train_n=800; eval_n=200; seed=42
ds = load_dataset(dataset_name)
ds_tr = ds["train"].shuffle(seed=seed).select(range(min(train_n, len(ds["train"]))))
ds_ev = ds["validation"].shuffle(seed=seed).select(range(min(eval_n, len(ds["validation"]))))

def preprocess(ex):
    ctx, q = ex["context"], ex["question"]
    ans = ex["answers"]["text"][0] if ex["answers"]["text"] else ""
    prompt = f"Use the context to answer concisely.\nContext: {ctx}\nQuestion: {q}\nAnswer:"
    model_in = tok(prompt, truncation=True, max_length=512)
    with tok.as_target_tokenizer():
        labels = tok(ans, truncation=True, max_length=64)
    model_in["labels"] = labels["input_ids"]
    model_in["id"] = ex["id"]
    return model_in

proc_tr = ds_tr.map(preprocess, remove_columns=ds_tr.column_names)
proc_ev = ds_ev.map(preprocess, remove_columns=ds_ev.column_names)
collator = DataCollatorForSeq2Seq(tokenizer=tok, model=base)

lora = LoraConfig(r=16, lora_alpha=32, target_modules=["q","k","v","o"], lora_dropout=0.05, bias="none", task_type="SEQ_2_SEQ_LM")
ft_model = get_peft_model(base, lora)
out_dir="artifacts/ft/flan_t5_small_lora"

# ✅ FIX: Removed evaluation_strategy and predict_with_generate
args = TrainingArguments(
    output_dir=out_dir,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-4,
    logging_steps=50,
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    bf16=torch.cuda.is_available(),
    fp16=False
)

trainer = Trainer(
    model=ft_model,
    args=args,
    train_dataset=proc_tr,
    eval_dataset=proc_ev,
    data_collator=collator,
    tokenizer=tok
)

trainer.train()
mets = trainer.evaluate()
print(mets)

trainer.model.save_pretrained(out_dir)
tok.save_pretrained(out_dir)

ft_cfg = {
    "base_model": base_model,
    "adapter_dir": out_dir,
    "dataset": dataset_name,
    "train_n": train_n,
    "eval_n": eval_n,
    "seed": seed
}
json.dump(ft_cfg, open("ft_config.json", "w"), indent=2)
print(json.dumps(ft_cfg, indent=2))

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/tmp/ipython-input-1156379243.py:45: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
50,0.638200
100,0.675800
150,0.559500
200,0.615100


{'eval_loss': 0.42560458183288574, 'eval_runtime': 3.3431, 'eval_samples_per_second': 59.824, 'eval_steps_per_second': 14.956, 'epoch': 1.0}
{
  "base_model": "google/flan-t5-small",
  "adapter_dir": "artifacts/ft/flan_t5_small_lora",
  "dataset": "squad",
  "train_n": 800,
  "eval_n": 200,
  "seed": 42
}


### 7) Merge & Evaluate (EM/F1 quick subset)

In [14]:

metric = evaluate.load("squad")
ft_loaded = AutoModelForSeq2SeqLM.from_pretrained(base_model)
ft_loaded = PeftModel.from_pretrained(ft_loaded, model_id="artifacts/ft/flan_t5_small_lora")
ft_loaded = ft_loaded.merge_and_unload()

pipe_ft = pipeline("text2text-generation", model=ft_loaded, tokenizer=tok,
                   max_new_tokens=64, device=0 if torch.cuda.is_available() else -1)

preds, refs = [], []
n = min(100, len(ds_ev))
for ex in ds_ev.select(range(n)):
    prompt = f"Use the context to answer concisely.\nContext: {ex['context']}\nQuestion: {ex['question']}\nAnswer:"
    pred = pipe_ft(prompt)[0]["generated_text"].strip()
    golds = ex["answers"]["text"] if ex["answers"]["text"] else [""]
    preds.append({"id": ex["id"], "prediction_text": pred})
    refs.append({"id": ex["id"], "answers": {"text": golds, "answer_start": ex["answers"]["answer_start"]}})
print(metric.compute(predictions=preds, references=refs))


Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


{'exact_match': 79.0, 'f1': 86.77539682539683}


### 8) Plug FT model into RAG and compare to Gemini

In [15]:

from langchain.llms.huggingface_pipeline import HuggingFacePipeline
llm_ft = HuggingFacePipeline(pipeline=pipe_ft)
qa_ft = RetrievalQA.from_chain_type(llm=llm_ft, chain_type="stuff", retriever=retriever)

qs = [
    "Summarize the dataset assumptions made in the uploaded materials.",
    "Identify two limitations or open problems noted in the documents."
]
print("\n=== RAG: Gemini ===")
for q in qs: print("\nQ:", q, "\nA:", RetrievalQA.from_chain_type(llm=llm_gemini, chain_type='stuff', retriever=retriever).invoke({'query': q})['result'][:800])
print("\n=== RAG: Fine-tuned FLAN-T5 ===")
for q in qs: print("\nQ:", q, "\nA:", qa_ft.invoke({'query': q})['result'][:800])



=== RAG: Gemini ===

Q: Summarize the dataset assumptions made in the uploaded materials. 
A: The provided materials do not explicitly state "dataset assumptions." Instead, they outline **requirements and guidelines for how datasets should be handled and documented**, particularly concerning their licensing and provenance.

These guidelines include:
*   Providing the service of the dataset's source.
*   When assets are released, including the license, copyright information, and terms of use in the package.
*   For popular datasets, paperswithcode.com/datasets is suggested as a resource for curated licenses and a licensing guide.
*   For existing datasets that are re-packaged, both the original license and the license of the derived asset (if it has changed) should be provided.

Q: Identify two limitations or open problems noted in the documents. 
A: Based on the provided documents, two limitations or open problems are:

1.  The **scope of the framework**.
2.  Any **strong assumptions*

### 9) Save/Update configs

In [16]:

rag_cfg = {
  "chunk_settings_tested":[{"chunk_size":500,"chunk_overlap":100},{"chunk_size":300,"chunk_overlap":50}],
  "embedding_models_tested":["sentence-transformers/all-MiniLM-L6-v2"],
  "llm":{"providers":["google-genai","hf-local"],"models":["gemini-2.5-flash","google/flan-t5-small (LoRA)"]},
  "retriever_k":4,
  "finetune": json.load(open("ft_config.json"))
}
json.dump(rag_cfg, open("rag_run_config.json","w"), indent=2)
print(json.dumps(rag_cfg, indent=2))


{
  "chunk_settings_tested": [
    {
      "chunk_size": 500,
      "chunk_overlap": 100
    },
    {
      "chunk_size": 300,
      "chunk_overlap": 50
    }
  ],
  "embedding_models_tested": [
    "sentence-transformers/all-MiniLM-L6-v2"
  ],
  "llm": {
    "providers": [
      "google-genai",
      "hf-local"
    ],
    "models": [
      "gemini-2.5-flash",
      "google/flan-t5-small (LoRA)"
    ]
  },
  "retriever_k": 4,
  "finetune": {
    "base_model": "google/flan-t5-small",
    "adapter_dir": "artifacts/ft/flan_t5_small_lora",
    "dataset": "squad",
    "train_n": 800,
    "eval_n": 200,
    "seed": 42
  }
}


### 10) Notes
- Use GPU runtime in Colab for fine-tuning.
- Keep keys out of version control.
- Increase dataset size/epochs for stronger results.